# Particle Filter Simulation Lab

In this lab, we’ll explore how particle filters can be used to estimate the position of a moving object.  
The filter will attempt to track the object based on noisy measurements and a motion model.

You’ll observe four different types of object motion:


| Motion Type | Motion Model Class | Description |
|-------------|--------------------|-------------|
| Circular    | `CircularMotion`   | Particles move in a circular path around a fixed center. |
| Elliptical  | `EllipseMotion`    | Particles follow an elliptical orbit, with different x and y radii. |
| Zig-Zag     | `ZigZagMotion`     | Particles move in straight lines and bounce off the edges of the world bounds. |
| Lissajous   | `LissajousMotion`  | Particles follow a complex, looped path defined by sine and cosine waves at different frequencies. |


We’ll use a single `ParticleFilter` class.  
Each particle moves according to a motion model (e.g., `CircularMotion`) and is reweighted using simulated sensor readings.

You’ll run simulations to visualize how accurately the filter can track the moving object.


## 0. Setup — Run This First

This cell loads all required libraries and sets the random seed.  
There’s nothing to edit here.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

rng = np.random.default_rng(3)

## 1. Motion Object Classes

These classes define how the "true" object moves in each simulation.  
Each one includes a `.step()` method to update the position and a `.noisy_measurement()` method to simulate sensor readings.

You won’t need to modify these classes.

In [ ]:
class CircularObject:
    def __init__(self, center=(0, 0), r=2.0, omega=0.1):
        self.cx, self.cy = center
        self.r = r
        self.omega = omega
        self.t = 0.0

    def step(self, dt=1.0):
        self.t += dt
        x = self.cx + self.r * np.cos(self.omega * self.t)
        y = self.cy + self.r * np.sin(self.omega * self.t)
        self.x, self.y = x, y
        return np.array([x, y])

    def noisy_measurement(self, std=0.4):
        pos = np.array([self.x, self.y])
        return np.array([
            np.linalg.norm(pos - lm) + rng.normal(0, std)
            for lm in LANDMARKS
        ])


class EllipseObject:
    def __init__(self, center=(0, 0), a=2.5, b=0.6, omega=0.05):
        self.center = np.array(center)
        self.a = a
        self.b = b
        self.omega = omega
        self.t = 0.0

    def step(self, dt=1.0):
        self.t += dt
        self.x = self.center[0] + self.a * np.cos(self.omega * self.t)
        self.y = self.center[1] + self.b * np.sin(self.omega * self.t)
        return np.array([self.x, self.y])

    def noisy_measurement(self, std=0.4):
        pos = np.array([self.x, self.y])
        return np.array([
            np.linalg.norm(pos - lm) + rng.normal(0, std)
            for lm in LANDMARKS
        ])


class ZigZagObject:
    def __init__(self, speed=0.1, bounds=(-4, 4, -4, 4)):
        self.pos = rng.uniform(-1, 1, size=2)
        angle = rng.uniform(0, 2 * np.pi)
        self.vel = speed * np.array([np.cos(angle), np.sin(angle)])
        self.speed = speed
        self.bounds = bounds

    def step(self, dt=1.0):
        self.pos += self.vel * dt

        # Bounce off walls
        x_min, x_max, y_min, y_max = self.bounds
        if self.pos[0] < x_min or self.pos[0] > x_max:
            self.vel[0] *= -1
            self.pos[0] = np.clip(self.pos[0], x_min, x_max)
        if self.pos[1] < y_min or self.pos[1] > y_max:
            self.vel[1] *= -1
            self.pos[1] = np.clip(self.pos[1], y_min, y_max)

        self.x, self.y = self.pos
        return self.pos

    def noisy_measurement(self, std=0.4):
        return np.array([
            np.linalg.norm(self.pos - lm) + rng.normal(0, std)
            for lm in LANDMARKS
        ])



class LissajousObject:
    def __init__(self, ax=2, bx=2, fx=0.1, fy=0.15, cx=0, cy=0):
        self.ax, self.bx = ax, bx
        self.fx, self.fy = fx, fy
        self.cx, self.cy = cx, cy
        self.t = 0.0

    def step(self, dt=1.0):
        self.t += dt
        self.x = self.cx + self.ax * np.sin(self.fx * self.t)
        self.y = self.cy + self.bx * np.cos(self.fy * self.t)
        return np.array([self.x, self.y])

    def noisy_measurement(self, std=0.4):
        pos = np.array([self.x, self.y])
        return np.array([
            np.linalg.norm(pos - lm) + rng.normal(0, std)
            for lm in LANDMARKS
        ])


## 2. Particle Filter Class

This is the main class that manages the particle filter.  
It handles:
- Initializing particles
- Predicting motion using a motion model
- Updating weights using sensor data
- Resampling particles
- Estimating the object’s position

Particles move according to the `motion_model.step(particle, dt)` method, which is passed into the constructor.


In [ ]:
class ParticleFilter:
    def __init__(self, n_particles, bounds, process_std=0.05, meas_std=0.4, motion_model=None):
        """
        Initialize the particle filter.

        Args:
            n_particles (int): Number of particles.
            bounds (tuple): (xmin, xmax, ymin, ymax) for particle initialization.
            process_std (float): Standard deviation of process noise.
            meas_std (float): Standard deviation of measurement noise.
            motion_model (object): Object with step() and jitter_particle() methods.
        """
        self.n = n_particles
        self.bounds = bounds
        self.process_std = process_std
        self.meas_std = meas_std
        self.motion_model = motion_model
        self.particles = self._init_particles(bounds)
        self.weights = np.ones(self.n) / self.n  # Start with uniform weights

    def _init_particles(self, b):
        """
        Initialize particles uniformly within bounds.

        Args:
            b (tuple): Bounds (xmin, xmax, ymin, ymax)

        Returns:
            list of dict: Each particle is a dictionary with a "pos" key.
        """

        raise NotImplementedError

    def apply_transition(self, dt=1.0):
        """
        Apply the motion model to each particle.

        TODO: Use self.motion_model to update each particle's position.
        """
        raise NotImplementedError

    def predict_measurement(self, particle):
        """
        Predict the distance measurements from a particle to landmarks.

        Args:
            particle (dict): A particle with "pos" key.

        Returns:
            np.ndarray: Predicted distances to each landmark.
        """
        # TODO: Return a vector of distances from pos to each landmark
        # Hint: Use np.linalg.norm for each landmark
        raise NotImplementedError

    def update_weights(self, measured_dists):
        """
        Update particle weights based on measurement likelihood.

        Args:
            measured_dists (np.ndarray): Actual sensor measurements to landmarks.

        TODO: Implement Gaussian likelihood to compare predicted and actual distances.
        """
        w = np.zeros(self.n)

        for i, p in enumerate(self.particles):
            w[i] = 1.0  # Replace with likelihood calculation

        # TODO: Normalize weights so they sum to 1

    def resample(self):
        """
        Resample particles based on their weights.

        TODO: Use np.random.choice to sample indices based on weights,
        then jitter new particles using the motion_model.
        """
        # TODO: Sample new indices from current particles
        # TODO: Apply jitter to copied particles
        # TODO: Reset weights to uniform
        raise NotImplementedError

    def estimate_state(self):
        """
        Estimate the most likely state from weighted particles.

        Returns:
            np.ndarray: Estimated position (x, y)
        """
        raise NotImplementedError

### 3. Motion Model Classes

These motion models define how each particle should move at each time step.  
They are **not** the same as the truth model — but should attempt to match its behavior.

Each model implements a `.step()` method that modifies the particle's position.

In [ ]:
class CircularMotion:
    def __init__(self, omega=0.1):
        self.omega = omega

    def step(self, particle, dt=1.0):
        if "theta" not in particle:
            particle["theta"] = rng.uniform(0, 2 * np.pi)
        if "center" not in particle:
            particle["center"] = rng.uniform(-2, 2, size=2)
        if "r" not in particle:
            particle["r"] = rng.uniform(1.0, 3.0)

        particle["theta"] += self.omega * dt
        theta = particle["theta"]
        center = particle["center"]
        r = particle["r"]
        particle["pos"] = center + r * np.array([np.cos(theta), np.sin(theta)])

    def jitter_particle(self, particle, std):
        particle["pos"] = particle["pos"].copy() + rng.normal(0, std, size=2)
        particle["center"] = particle["center"].copy() + rng.normal(0, std, size=2)
        particle["r"] = max(0.1, particle["r"] + rng.normal(0, std))
        return particle




class EllipseMotion:
    def __init__(self, center=(0, 0), a=2.5, b=0.6, omega=0.05):
        self.a = a
        self.b = b
        self.omega = omega

    def step(self, particle, dt=1.0):
        if "theta" not in particle:
            particle["theta"] = rng.uniform(0, 2 * np.pi)

        if "center" not in particle:
            particle["center"] = rng.uniform(-2, 2, size=2)

        if "a" not in particle:
            particle["a"] = rng.uniform(0, 3)
            particle["b"] = rng.uniform(0, 3)

        particle["theta"] += self.omega * dt
        angle = particle["theta"]
        cx, cy = particle["center"]
        particle["pos"] = np.array([
            cx + self.a * np.cos(angle),
            cy + self.b * np.sin(angle)
        ])

    def jitter_particle(self, particle, std):
        particle["pos"] = particle["pos"].copy() + rng.normal(0, std, size=2)
        if "center" in particle:
            particle["center"] = particle["center"].copy() + rng.normal(0, std, size=2)
        if "a" in particle:
            particle["a"] += rng.normal(0, std)
        if "b" in particle:
            particle["b"] += rng.normal(0, std)
        return particle




class ZigZagMotion:
    def __init__(self, speed=0.1, bounds=(-4, 4, -4, 4)):
        self.speed = speed
        self.bounds = bounds

    def step(self, particle, dt=1.0):
        if "vel" not in particle:
            angle = rng.uniform(0, 2 * np.pi)
            particle["vel"] = self.speed * np.array([np.cos(angle), np.sin(angle)])

        particle["pos"] += particle["vel"] * dt

        x_min, x_max, y_min, y_max = self.bounds
        if particle["pos"][0] < x_min or particle["pos"][0] > x_max:
            particle["vel"][0] *= -1
            particle["pos"][0] = np.clip(particle["pos"][0], x_min, x_max)
        if particle["pos"][1] < y_min or particle["pos"][1] > y_max:
            particle["vel"][1] *= -1
            particle["pos"][1] = np.clip(particle["pos"][1], y_min, y_max)

    def jitter_particle(self, particle, std):
        particle["pos"] = particle["pos"].copy() + rng.normal(0, std, size=2)
        angle = np.arctan2(particle["vel"][1], particle["vel"][0])
        angle += rng.normal(0, std)
        speed = np.linalg.norm(particle["vel"])
        particle["vel"] = speed * np.array([np.cos(angle), np.sin(angle)])

        return particle



class LissajousMotion:
    def __init__(self, ax=2, bx=2, fx=0.1, fy=0.15, cx=0, cy=0):
        self.ax, self.bx = ax, bx
        self.fx, self.fy = fx, fy
        self.cx, self.cy = cx, cy

    def step(self, particle, dt=1.0):
        if "t" not in particle:
            particle["t"] = rng.uniform(0, 100)

        particle["t"] += dt
        t = particle["t"]
        particle["pos"] = np.array([
            self.cx + self.ax * np.sin(self.fx * t),
            self.cy + self.bx * np.cos(self.fy * t)
        ])

    def jitter_particle(self, particle, std):
        particle["pos"] = particle["pos"].copy() + rng.normal(0, std, size=2)
        particle["t"] += rng.normal(0, std)
        return particle




## 4. Visualization — `run_simulation()`

This function runs the simulation and shows how the particles move over time.

It takes:
- A `truth_obj` (one of the moving object classes)
- A `ParticleFilter` instance
- The number of time steps to simulate

Use this to test each motion model below.


In [ ]:
def run_simulation(truth_obj, pf, T=100):
    BOUNDS = pf.bounds
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set(xlim=(BOUNDS[0], BOUNDS[1]), ylim=(BOUNDS[2], BOUNDS[3]),
           xlabel='x', ylabel='y', title='Particle Filter Tracking', aspect='equal')
    scat = ax.scatter([], [], s=8, alpha=0.2)
    truth_dot, = ax.plot([], [], 'ko', label='Truth')
    est_star, = ax.plot([], [], 'r*', ms=10, label='Estimate')
    landmark_arr = np.array(LANDMARKS)
    landmark_dots = ax.scatter(landmark_arr[:, 0], landmark_arr[:, 1],
                              marker='x', color='blue', s=80, label='Landmarks')

    ax.legend(loc='upper left')

    def init():
        scat.set_offsets(np.array([p["pos"] for p in pf.particles]))
        scat.set_sizes(np.full(len(pf.particles), 8))
        truth_dot.set_data([], [])
        est_star.set_data([], [])
        return scat, truth_dot, est_star


    def step(frame):
        if frame == 0:
            scat.set_offsets(np.array([p["pos"] for p in pf.particles]))
            scat.set_sizes(np.full(len(pf.particles), 8))
            truth_dot.set_data([], [])
            est_star.set_data([], [])
            return scat, truth_dot, est_star

        truth_pos = truth_obj.step()
        meas = truth_obj.noisy_measurement(pf.meas_std)

        # TODO: move particles
        # TODO: update weights
        # TODO: resample

        scat.set_offsets(np.array([p["pos"] for p in pf.particles]))
        truth_dot.set_data([truth_pos[0]], [truth_pos[1]])
        est = pf.estimate_state()
        est_star.set_data([est[0]], [est[1]])
        return scat, truth_dot, est_star


    ani = animation.FuncAnimation(fig, step, frames=T, init_func=init, interval=100, blit=False)
    return HTML(ani.to_jshtml())


## 5. Common Parameters

These global variables control how the particle filter behaves during each simulation:

| Parameter       | Variable Name     | Description |
|----------------|-------------------|-------------|
| Number of particles | `N_PARTICLES`   | How many particles are used to represent the belief distribution. More particles = better accuracy but slower performance. |
| World bounds    | `BOUNDS`          | The (x, y) limits for particle initialization: `(xmin, xmax, ymin, ymax)`. |
| Process noise   | `PROCESS_STD`     | Controls how much randomness is added to each particle’s motion at each time step. Simulates uncertainty in movement. |
| Measurement noise | `MEAS_STD`     | Standard deviation of the noise added to sensor measurements. Simulates uncertainty in the distance readings. |
| Landmark positions | `LANDMARKS`   | Known fixed points in the environment. Each sensor reading represents the (noisy) distance to one of these landmarks. |

You can try adjusting these values to explore how the filter's performance changes.


In [ ]:
N_PARTICLES = 500
BOUNDS = (-4, 4, -4, 4)
PROCESS_STD = 0.05
MEAS_STD = 0.4
LANDMARKS = [np.array([0, 2]), np.array([0, -2]), np.array([1, 1])]

# 6. Test the Particle Filter

## Circular Motion

In this simulation, the object moves in a circle.

- The truth is modeled by the `CircularObject` class.
- Particles move using the `CircularMotion` model.

Run the simulation below and observe how well the particle cloud tracks the true position.


In [ ]:
truth = CircularObject()
motion = CircularMotion(omega=0.1)
pf = ParticleFilter(N_PARTICLES, BOUNDS, PROCESS_STD, MEAS_STD, motion_model=motion)
run_simulation(truth, pf)

## Lissajous Motion

In this simulation, the object follows a Lissajous curve.

- The truth is modeled by the `LissajousObject` class.
- Particles move using the `LissajousMotion` model.

Run the simulation below and observe how well the particle cloud tracks the true position.


In [ ]:
truth = LissajousObject()
motion = LissajousMotion()
pf = ParticleFilter(N_PARTICLES, BOUNDS, PROCESS_STD, MEAS_STD, motion_model=motion)
run_simulation(truth, pf)

## Elliptical Motion

In this simulation, the object moves in an ellipse.

- The truth is modeled by the `EllipseObject` class.
- Particles move using the `EllipseMotion` model.

Run the simulation below and observe how well the particle cloud tracks the true position.


In [ ]:
truth = EllipseObject()
motion = EllipseMotion()
pf = ParticleFilter(N_PARTICLES, BOUNDS, PROCESS_STD, MEAS_STD, motion_model=motion)
run_simulation(truth, pf)

## Zig-Zag Motion

In this simulation, the object moves with random bounces inside a bounded box.

- The truth is modeled by the `ZigZagObject` class.
- Particles move using the `ZigZagMotion` model.

Run the simulation below and observe how well the particle cloud tracks the true position.


In [ ]:
truth = ZigZagObject()
motion = ZigZagMotion()
pf = ParticleFilter(N_PARTICLES, BOUNDS, PROCESS_STD, MEAS_STD, motion_model=motion)
run_simulation(truth, pf)

# 7. Investigation

Use these questions to explore how your particle filter behaves under different conditions. Modify parameters, run experiments, and analyze your results.

---

1. **Effect of Number of Particles**
   - Try different values: `n_particles = 10, 100, 500, 1000`.
   - How does increasing the number of particles affect:
     - Accuracy of tracking?
     - Responsiveness to changes?
     - Computational performance?

2. **Process vs. Measurement Noise**
   - Keep `meas_std` constant and vary `process_std`. Then swap.
   - What happens to particle spread and tracking accuracy?
   - Which type of noise has a stronger impact on performance?

3. **Effect of Landmark Placement**
   - Run the simulation with different landmark configurations:
     - All landmarks near the center
     - All in one corner
     - Evenly spaced in a grid
     - Spread in a circle around the tracking area
   - How does the placement affect the filter's ability to accurately track the object?

4. **Ambiguity from Symmetry**
   - Try placing landmarks in a symmetric pattern (e.g., corners of a square).
   - Does this create ambiguity in the estimated position?
   - Does it vary by motion model?
   - What does the particle distribution look like when symmetry causes confusion?

5. **Minimal Landmark Set**
   - What is the minimum number of landmarks needed to track the object reliably? Does it differ by motion model?
   - Try removing landmarks one at a time — at what point does tracking noticeably degrade?

6. **Landmark Spread and Accuracy**
   - Compare simulations with:
     - Landmarks clustered close together
     - Landmarks widely spaced
   - Which configuration leads to:
     - Faster convergence?
     - More stable estimation?
   - Why?


In [ ]:
N_PARTICLES = 500
BOUNDS = (-4, 4, -4, 4)
PROCESS_STD = 0.05
MEAS_STD = 0.4
LANDMARKS = [np.array([0, 2]), np.array([0, -2]), np.array([1, 1])]

truth = CircularObject()
motion = CircularMotion(omega=0.1)
pf = ParticleFilter(N_PARTICLES, BOUNDS, PROCESS_STD, MEAS_STD, motion_model=motion)
run_simulation(truth, pf)

### Extensions

7. **Mismatched Motion Models**
   - Track a `CircularObject` using a `ZigZagMotion` particle model.
   - How does this mismatch affect performance?
   - Can the filter still track the object?

8. **Compare Motion Models**
   - Try different combinations of truth models and particle motion models.
     - Example: `LissajousObject` + `CircularMotion`
     - Which combinations perform best? Why?

9. **Initial Position**
   - Initialize all particles in the same spot instead of uniformly across the grid.
   - Can the filter recover over time?
   - How many steps does it take to "lock on"?

10. **Sparse Measurements**
   - Modify the Particle Filter to track how many steps and to take measurements only every 2 or 5 steps.
   - How does this affect filter accuracy and stability?

11. **Track Estimation Error**
   - Modify run_simulation to compute the Euclidean distance between the estimated and true positions at each timestep.
   - Plot error vs. time for different parameter settings.
   - Which configuration produces the smallest long-term error?

12. **Adaptive Noise**
   - Modify the Particle Filter to reduce `meas_std` over time.
   - Does this help when the filter is already close to the object?

10. **Improved Resampling**
    - Implement stratified or systematic resampling instead of naive random choice.
    - Does this reduce particle degeneracy and improve tracking?


In [ ]:
N_PARTICLES = 500
BOUNDS = (-4, 4, -4, 4)
PROCESS_STD = 0.05
MEAS_STD = 0.4
LANDMARKS = [np.array([0, 2]), np.array([0, -2]), np.array([1, 1])]

truth = CircularObject()
motion = CircularMotion(omega=0.1)
pf = ParticleFilter(N_PARTICLES, BOUNDS, PROCESS_STD, MEAS_STD, motion_model=motion)
run_simulation(truth, pf)